In [1]:
from ultralytics import YOLO
import cv2
import numpy as np
import json

In [7]:
def detect_object(image_path):
    # Read the image
    image = cv2.imread(image_path)
    
    # Convert to HSV color space
    hsv = cv2.cvtColor(image, cv2.COLOR_BGR2HSV)
    
    # Define range for red color
    lower_red1 = np.array([0, 100, 100])
    upper_red1 = np.array([10, 255, 255])
    lower_red2 = np.array([160, 100, 100])
    upper_red2 = np.array([180, 255, 255])
    
    # Create masks for red color
    mask1 = cv2.inRange(hsv, lower_red1, upper_red1)
    mask2 = cv2.inRange(hsv, lower_red2, upper_red2)
    mask = cv2.bitwise_or(mask1, mask2)
    
    # Find contours
    contours, _ = cv2.findContours(mask, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)

    # Image height and width
    height, width = image.shape[:2]
    
    if contours:
        # Process largest contour
        largest_contour = max(contours, key=cv2.contourArea)
        area = cv2.contourArea(largest_contour)
        
        if area > 500:
            # Get bounding box
            x, y, w, h = cv2.boundingRect(largest_contour)
            
            # Calculate centers using both methods
            box_center_x = x + w//2
            box_center_y = y + h//2
            
            # Calculate moment-based center
            M = cv2.moments(largest_contour)
            if M["m00"] != 0:
                # Floating point centers
                moment_center_x_float = M["m10"] / M["m00"]
                moment_center_y_float = M["m01"] / M["m00"]

                # Integer centers for drawing
                moment_center_x = int(moment_center_x_float)
                moment_center_y = int(moment_center_y_float)
                
                moment_center_x = int(M["m10"] / M["m00"])
                moment_center_y = int(M["m01"] / M["m00"])
                
                # Draw both centers for comparison
                cv2.circle(image, (box_center_x, box_center_y), 5, (0, 255, 0), -1)  # Green for box center
                cv2.circle(image, (moment_center_x, moment_center_y), 5, (255, 0, 0), -1)  # Blue for moment center
                cv2.rectangle(image, (x, y), (x + w, y + h), (0, 255, 0), 2)
                
                cv2.imwrite(image_detection, image)
                
                # Return box center coordinates instead of moment-based
                # return box_center_x, box_center_y, x, y, w, h, height, width
                return moment_center_x, moment_center_y, box_center_x, box_center_y, x, y, w, h, height, width
    
    return None, None, None, None, None, None, None, None, None, None

In [8]:
def detect_objects(image_path):
    # Read the image
    image = cv2.imread(image_path)
    
    # Convert to HSV color space
    hsv = cv2.cvtColor(image, cv2.COLOR_BGR2HSV)
    
    # Define range for red color
    lower_red1 = np.array([0, 100, 100])
    upper_red1 = np.array([10, 255, 255])
    lower_red2 = np.array([160, 100, 100])
    upper_red2 = np.array([180, 255, 255])
    
    # Create masks for red color
    mask1 = cv2.inRange(hsv, lower_red1, upper_red1)
    mask2 = cv2.inRange(hsv, lower_red2, upper_red2)
    mask = cv2.bitwise_or(mask1, mask2)
    
    # Find contours
    contours, _ = cv2.findContours(mask, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)
    
    # Image height and width
    height, width = image.shape[:2]
    
    # List to store all detected boxes
    detected_boxes = []
    
    for contour in contours:
        area = cv2.contourArea(contour)
        
        if area > 500:  # Minimum area threshold
            # Get bounding box
            x, y, w, h = cv2.boundingRect(contour)
            
            # Calculate box center
            box_center_x = x + w//2
            box_center_y = y + h//2
            
            # Calculate moment-based center
            M = cv2.moments(contour)
            if M["m00"] != 0:
                moment_center_x = int(M["m10"] / M["m00"])
                moment_center_y = int(M["m01"] / M["m00"])
                
                # Draw centers and rectangle
                cv2.circle(image, (box_center_x, box_center_y), 5, (0, 255, 0), -1)  # Green for box center
                cv2.circle(image, (moment_center_x, moment_center_y), 5, (255, 0, 0), -1)  # Blue for moment center
                cv2.rectangle(image, (x, y), (x + w, y + h), (0, 255, 0), 2)
                
                # Store detection results
                detected_boxes.append({
                    'moment_center': (moment_center_x, moment_center_y),
                    'box_center': (box_center_x, box_center_y),
                    'bbox': (x, y, w, h)
                })
    
    cv2.imwrite(image_detection, image)
    
    return detected_boxes, height, width

In [23]:
def detect_objects(image_path):
    # Read the image
    image = cv2.imread(image_path)
    
    # Convert to HSV color space
    hsv = cv2.cvtColor(image, cv2.COLOR_BGR2HSV)
    
    # Define range for red color
    lower_red1 = np.array([0, 100, 100])
    upper_red1 = np.array([10, 255, 255])
    lower_red2 = np.array([160, 100, 100])
    upper_red2 = np.array([180, 255, 255])
    
    # Create masks for red color
    mask1 = cv2.inRange(hsv, lower_red1, upper_red1)
    mask2 = cv2.inRange(hsv, lower_red2, upper_red2)
    mask = cv2.bitwise_or(mask1, mask2)
    
    # Find contours
    contours, _ = cv2.findContours(mask, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)
    
    # Image height and width
    height, width = image.shape[:2]
    
    # List to store all detected boxes
    detected_boxes = []
    
    for contour in contours:
        area = cv2.contourArea(contour)
        
        if area > 500:  # Minimum area threshold
            # Calculate moment-based center first
            M = cv2.moments(contour)
            if M["m00"] != 0:
                moment_center_x = int(M["m10"] / M["m00"])
                moment_center_y = int(M["m01"] / M["m00"])
                
                # Get bounding box (for drawing only)
                x, y, w, h = cv2.boundingRect(contour)
                
                # Convert to Unity coordinates using moment center
                unity_x = (moment_center_x - 245) * 0.020408163 + 14
                
                # Draw visuals
                cv2.circle(image, (moment_center_x, moment_center_y), 5, (255, 0, 0), -1)  # Blue for moment center
                cv2.rectangle(image, (x, y), (x + w, y + h), (0, 255, 0), 2)
                
                # Store detection results
                detected_boxes.append({
                    'moment_center': (moment_center_x, moment_center_y),
                    'unity_position': unity_x,
                    'bbox': (x, y, w, h)
                })
    
    cv2.imwrite(image_detection, image)
    
    return detected_boxes, height, width

In [10]:
def detect_objects(image_path, image_detection):
    # Read the image
    image = cv2.imread(image_path)
    
    # Convert to HSV color space
    hsv = cv2.cvtColor(image, cv2.COLOR_BGR2HSV)
    
    # Define range for red color
    lower_red1 = np.array([0, 100, 100])
    upper_red1 = np.array([10, 255, 255])
    lower_red2 = np.array([160, 100, 100])
    upper_red2 = np.array([180, 255, 255])
    
    # Define range for blue color
    lower_blue = np.array([90, 100, 100])
    upper_blue = np.array([130, 255, 255])
    
    # Create masks for both colors
    mask1 = cv2.inRange(hsv, lower_red1, upper_red1)
    mask2 = cv2.inRange(hsv, lower_red2, upper_red2)
    red_mask = cv2.bitwise_or(mask1, mask2)
    blue_mask = cv2.inRange(hsv, lower_blue, upper_blue)
    
    # Combine masks
    combined_mask = cv2.bitwise_or(red_mask, blue_mask)
    
    # Find contours
    contours, _ = cv2.findContours(combined_mask, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)
    
    # Image height and width
    height, width = image.shape[:2]
    
    # List to store all detected boxes
    detected_boxes = []
    
    for contour in contours:
        area = cv2.contourArea(contour)
        
        if area > 500:  # Minimum area threshold
            # Get bounding box
            x, y, w, h = cv2.boundingRect(contour)
            
            # Calculate box center
            box_center_x = x + w//2
            box_center_y = y + h//2
            
            # Check if box is red or blue
            is_red = cv2.countNonZero(red_mask[y:y+h, x:x+w]) > 0
            is_blue = cv2.countNonZero(blue_mask[y:y+h, x:x+w]) > 0
            
            color = 'red' if is_red else 'blue' if is_blue else 'unknown'
            
            # # Convert to Unity coordinates for both colors
            
            
            # Calculate moment center
            M = cv2.moments(contour)
            if M["m00"] != 0:
                moment_center_x = int(M["m10"] / M["m00"])
                moment_center_y = int(M["m01"] / M["m00"])

                # unity_x = (box_center_x - 248) * 0.020689655 + 14
                unity_y = (moment_center_y - 264) * (-0.0235)
                
                # Draw visuals
                rect_color = (0, 0, 255) if color == 'red' else (255, 0, 0)  # Red or Blue
                cv2.circle(image, (box_center_x, box_center_y), 5, (0, 255, 0), -1)  # Green for box center
                cv2.rectangle(image, (x, y), (x + w, y + h), (0, 255, 0), 2)
                
                # Store detection results
                detected_boxes.append({
                    'moment_center': (moment_center_x, moment_center_y),
                    'box_center': (box_center_x, box_center_y),
                    'bbox': (x, y, w, h),
                    'color': color,
                    'unity_position': unity_y
                })
    
    cv2.imwrite(image_detection, image)
    
    return detected_boxes, height, width

In [11]:
image_name = "image1"
image_path = image_name + ".png"
image_detection = image_name + "detection.png"
# moment_center_x, moment_center_y, box_center_x, box_center_y, edge_x, edge_y, w, h, image_height, image_width = detect_objects(image_path)
detected_boxes, image_height, image_width = detect_objects(image_path, image_detection)

In [12]:
detected_boxes

[{'moment_center': (388, 434),
  'box_center': (388, 434),
  'bbox': (350, 388, 77, 93),
  'color': 'blue',
  'unity_position': -3.995},
 {'moment_center': (387, 264),
  'box_center': (388, 264),
  'bbox': (350, 230, 77, 69),
  'color': 'red',
  'unity_position': -0.0}]

In [58]:
for box in detected_boxes:
    print(box)

{'moment_center': (473, 345), 'box_center': (473, 345), 'bbox': (443, 308, 61, 74), 'color': 'blue', 'unity_position': -3.9983999999999997}
{'moment_center': (472, 209), 'box_center': (473, 210), 'bbox': (443, 183, 61, 54), 'color': 'red', 'unity_position': -0.0}


# How to calculate the relationship between Python position and Unity position
https://claude.ai/chat/b57bdb8d-0542-4a2c-955c-9d54754c0967%2014
- Python (x = 14) -> Unity (x = 288)
- Python (x = 16) -> Unity (x = 371)
- Python (x = 18) -> Unity (x = 455)
- Python (x = 20) -> Unity (x = 538)


- coefficent_relationship = from 18 to 20 (2 unit) = (538 - 455) / 2 = 41.5
- Python_X = (Unity_X * coefficent_relationship) + some_base_offset
- => some_base_offset = -293
- Python_X = (Unity_X * 41.5) - 293